In [1]:
import pandas as pd
import json
import time
import os
import re

In [2]:
import sys
myargs = sys.argv
if '-f' in myargs:
    try: FILE_NAME = myargs[2]
    except Exception as e:
        print(e)
        FILE_NAME = 'template'
else:
    FILE_NAME = 'template'
    os.system("cp ./template.csv ./temp/")
# if testing on jupyter notebook
if FILE_NAME[0] == "/":
    FILE_NAME = 'template'
    os.system("cp ./template.csv ./temp/")

# generate ID
import random
def generate_ID():
    c = ["".join([random.choice('01234567890abcdef') for _ in range(4)]) 
         for _ in range(8)]
    return c[0]+c[1]+"-"+c[2]+"-"+c[3]+"-"+c[4]+"-"+c[5]+c[6]+c[7]

generate_ID()

'e2544e55-9fb1-48ff-f609-0d3670bcc120'

In [3]:
# copy template
os.system("rm -r template")
os.system("cp -r stencil temp/{}".format(FILE_NAME))

0

In [4]:
df = pd.read_csv('temp/{}.csv'.format(FILE_NAME), header=0)
df

,Unnamed: 0,INTENT_NAME,INPUT_CONTEXT,Unnamed: 3,OUTPUT_CONTEXT,USER_SAYS,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,RESPONSES.1,RESPONSES.2
0,0,Default Fallback Intent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Can you say that again?,"Sorry, I didn't get that. Can you rephrase?","Sorry, what was that?",One more time?,What was that?,Say that one more time?,I didn't get that. Can you repeat?,"I missed that, say that again?",NaN,NaN
1,1,Default Welcome Intent,NaN,NaN,"3, init",just going to say hi,heya,hello hi,howdy,hey there,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,make chatbot,NaN,NaN,NaN,i want chatbot,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,make coffee,thirsty,NaN,"0, thirsty",i want coffee,lim kopi ai mai,kopi siu dai gam xia,NaN,NaN,...,now converting coffee into code,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,make love,rich,trouble,"0, trouble",给我一个吻,i want love,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,this is a second response bubble,"{""trap"": ""trying to break this system""}"
5,5,make money,NaN,NaN,"2, thirsty",i want money,bo lui ah,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,make tea,thirsty,NaN,"10, woke",i want tea,lim tehsi,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,make trouble,NaN,NaN,NaN,kua simi kua,le kua si mi,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# get column number for manipulation next
col_num_INTENT_NAME = df.columns.get_loc("INTENT_NAME")
col_num_IN = df.columns.get_loc("INPUT_CONTEXT")
col_num_OUT = df.columns.get_loc("OUTPUT_CONTEXT")
col_num_USER = df.columns.get_loc("USER_SAYS")
cols_name_RESP = [col for col in df if col.startswith('RESPONSES')]
cols_num_RESP = []
for col_name_RESP in cols_name_RESP:
    cols_num_RESP.append(df.columns.get_loc(col_name_RESP))
cols_num_RESP.append(len(df.columns))
col_num_total = len(df.columns)

In [6]:
def rm_nan(list_of_list):
    '''remove nan from list of list'''
    return [[x for x in list_ if x == x] for list_ in list_of_list]
    
df['INPUT_CONTEXT_L'] = rm_nan(df.iloc[:,col_num_IN:col_num_OUT].values.tolist())
df['OUTPUT_CONTEXT_L'] = rm_nan(df.iloc[:,col_num_OUT:col_num_USER].values.tolist())
df['USER_SAYS_L'] = rm_nan(df.iloc[:,col_num_USER:cols_num_RESP[0]].values.tolist())
for i,col_num_RESP in enumerate(cols_num_RESP[:-1]):
    df['RESPONSES_L_{}'.format(i)] = rm_nan(df.iloc[:,col_num_RESP:cols_num_RESP[i+1]].values.tolist())

In [7]:
# combine columns, ignoring nan
# https://stackoverflow.com/questions/45787782/combine-multiple-columns-in-pandas-excluding-nans
# df['INPUT_CONTEXT_L'] = df.iloc[:,col_num_IN:col_num_OUT].apply(lambda x: list(x.dropna()), axis=1).values.tolist()
# df['OUTPUT_CONTEXT_L'] = df.iloc[:,col_num_OUT:col_num_USER].apply(lambda x: list(x.dropna()), axis=1).values.tolist()
# df['USER_SAYS_L'] = df.iloc[:,col_num_USER:col_num_RESP].apply(lambda x: list(x.dropna()), axis=1).values.tolist()
# df['RESPONSES_L'] = df.iloc[:,col_num_RESP:col_num_total].apply(lambda x: list(x.dropna()), axis=1).values.tolist()
# I don't understand why line 1 works but line 3 and 4 don't work. WTF.

In [8]:
df

,Unnamed: 0,INTENT_NAME,INPUT_CONTEXT,Unnamed: 3,OUTPUT_CONTEXT,USER_SAYS,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 31,Unnamed: 32,RESPONSES.1,RESPONSES.2,INPUT_CONTEXT_L,OUTPUT_CONTEXT_L,USER_SAYS_L,RESPONSES_L_0,RESPONSES_L_1,RESPONSES_L_2
0,0,Default Fallback Intent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,I didn't get that. Can you repeat?,"I missed that, say that again?",NaN,NaN,[],[],[],"[I didn't get that. Can you say it again?, I m...",[],[]
1,1,Default Welcome Intent,NaN,NaN,"3, init",just going to say hi,heya,hello hi,howdy,hey there,...,NaN,NaN,NaN,NaN,[],"[3, init]","[just going to say hi, heya, hello hi, howdy, ...",[Welcome to ABC law partners. \nWe can help ex...,[],[]
2,2,make chatbot,NaN,NaN,NaN,i want chatbot,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,[],[],[i want chatbot],"[皮卡丘似乎对你妈妈感兴趣, بيكاتشو يبدو مهتما بأمك, пикачу...",[],[]
3,3,make coffee,thirsty,NaN,"0, thirsty",i want coffee,lim kopi ai mai,kopi siu dai gam xia,NaN,NaN,...,NaN,NaN,NaN,NaN,[thirsty],"[0, thirsty]","[i want coffee, lim kopi ai mai, kopi siu dai ...","[half full drink up!, bo ta bo lam pah, bottom...",[],[]
4,4,make love,rich,trouble,"0, trouble",给我一个吻,i want love,NaN,NaN,NaN,...,NaN,NaN,this is a second response bubble,"{""trap"": ""trying to break this system""}","[rich, trouble]","[0, trouble]","[给我一个吻, i want love]",[<3],[this is a second response bubble],"[{""trap"": ""trying to break this system""}]"
5,5,make money,NaN,NaN,"2, thirsty",i want money,bo lui ah,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,[],"[2, thirsty]","[i want money, bo lui ah]",[huat ah],[],[]
6,6,make tea,thirsty,NaN,"10, woke",i want tea,lim tehsi,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,[thirsty],"[10, woke]","[i want tea, lim tehsi]","[slowly drink, take your time]",[],[]
7,7,make trouble,NaN,NaN,NaN,kua simi kua,le kua si mi,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,[],[],"[kua simi kua, le kua si mi]",[come fight la],[],[]


In [9]:
# # for line in rows of excel
# intent_df_row = None 
for index, row in df.iterrows():  # CURRENTLY ONLY PROCESSING ONE ROW
    if index == 4:
        intent_df_row = row
#     break

In [10]:
intent_name = intent_df_row['INTENT_NAME']
input_contexts = intent_df_row['INPUT_CONTEXT_L']
output_contexts = [[y.strip() for y in x.split(',')]
                   for x in intent_df_row['OUTPUT_CONTEXT_L']]
training_phrases = intent_df_row['USER_SAYS_L']
responses = [intent_df_row['RESPONSES_L_{}'.format(i)] for i,x in enumerate(cols_num_RESP[:-1])]
responses = [x for x in responses if x != []]

In [11]:
# define a list/dictionary for {intent_name}.json
intent_jsonfile = {}
intent_jsonfile["id"] = generate_ID() # to randomly generate
intent_jsonfile["name"] = intent_name
intent_jsonfile["auto"] = True # will change to lowercase when converted to jsonfile
intent_jsonfile["contexts"] = input_contexts

intent_jsonfile_responses = {}
intent_jsonfile_responses["resetContexts"] = False
intent_jsonfile_responses["affectedContexts"] = []

for output_context in output_contexts:
    affectedContext = {}
    affectedContext["name"] = output_context[1]
    affectedContext["parameters"] = {}
    affectedContext["lifespan"] = output_context[0]
    intent_jsonfile_responses["affectedContexts"].append(affectedContext)
    
intent_jsonfile_responses["parameters"] = []
intent_jsonfile_responses["messages"] = []  # probably allow for list of messages? yes

for response_bubble in responses:
    message = {}
    message["type"] = 0
    message["lang"] = "en"
    message["speech"] = response_bubble
    intent_jsonfile_responses["messages"].append(message)

intent_jsonfile_responses["defaultResponsePlatforms"] = []
intent_jsonfile_responses["speech"] = []
intent_jsonfile["responses"] = [intent_jsonfile_responses]  #

intent_jsonfile["priority"] = 500000
intent_jsonfile["webhookUsed"] = False
intent_jsonfile["webhookForSlotFilling"] = False
intent_jsonfile["lastUpdate"] = int(time.time())
intent_jsonfile["fallbackIntent"] =  False
intent_jsonfile["events"] = []

# 'messages': [{'speech': '<3', 'lang': 'en', 'type': 0}, 
#              {'speech': 'this is a second response bubble', 'lang': 'en', 'type': 0}, 
#              {'payload': {'trap': 'trying to break this system'}, 'lang': 'en', 'type': 4}]

In [12]:
print(json.dumps(intent_jsonfile))

{"contexts": ["rich", "trouble"], "fallbackIntent": false, "webhookUsed": false, "name": "make love", "auto": true, "priority": 500000, "events": [], "lastUpdate": 1558419867, "id": "280a0e23-e83d-8d32-af07-8370955f886f", "responses": [{"messages": [{"type": 0, "speech": ["<3"], "lang": "en"}, {"type": 0, "speech": ["this is a second response bubble"], "lang": "en"}, {"type": 0, "speech": ["{\"trap\": \"trying to break this system\"}"], "lang": "en"}], "parameters": [], "affectedContexts": [{"parameters": {}, "lifespan": "0", "name": "trouble"}], "resetContexts": false, "defaultResponsePlatforms": [], "speech": []}], "webhookForSlotFilling": false}


{
  "id": "f60c8a1f-2911-448b-90c6-facd27259ed0",
  "name": "training phrases test",
  "auto": true,
  "contexts": [
    "inputcontexttest"
  ],
  "responses": [
    {
      "resetContexts": false,
      "affectedContexts": [
        {
          "name": "outputcontexttest",
          "parameters": {},
          "lifespan": 999
        }
      ],
      "parameters": [],
      "messages": [
        {
          "type": 0,
          "lang": "en",
          "speech": [
            "sample response one",
            "sample response two",
            "sample response three"
          ]
        }
      ],
      "defaultResponsePlatforms": {},
      "speech": []
    }
  ],
  "priority": 500000,
  "webhookUsed": false,
  "webhookForSlotFilling": false,
  "lastUpdate": 1548814094,
  "fallbackIntent": false,
  "events": []
}

In [13]:
# define a list/dictionary for {intent_usersays_name}.json
intent_usersays_jsonfile = []
for training_phrase in training_phrases:
    user_say = {}
    user_say["id"] = generate_ID()
    user_say["data"] = []  # no entities for now and the near future
    snippet = {}
    snippet["text"] = training_phrase
    snippet["userDefined"] = False
    user_say["data"] = [snippet]
    
    user_say["isTemplate"] = False
    user_say["count"] = 0
    user_say["updated"] = int(time.time())
    intent_usersays_jsonfile.append(user_say)

In [14]:
print(json.dumps(intent_usersays_jsonfile))

[{"isTemplate": false, "count": 0, "id": "cd11c254-8943-c405-4dee-9a08d6e003dd", "data": [{"userDefined": false, "text": "\u7ed9\u6211\u4e00\u4e2a\u543b"}], "updated": 1558419867}, {"isTemplate": false, "count": 0, "id": "a854cc4f-897f-771b-3a01-05512c200392", "data": [{"userDefined": false, "text": "i want love"}], "updated": 1558419867}]


[
  {
    "id": "f65bbd35-97d6-4862-bd56-5e360dbe7d08",
    "data": [
      {
        "text": "training phrase ",
        "userDefined": false
      },
      {
        "text": "three",
        "meta": "@sys.ignore",
        "userDefined": false
      }
    ],
    "isTemplate": false,
    "count": 0,
    "updated": 1548814041
  }
]

In [15]:
intent_name = re.sub(r"[^a-zA-Z0-9- ]", '_', intent_name)

with open('temp/{}/intents/{}.json'.format(FILE_NAME, intent_name), 'w') as outfile:
    json.dump(intent_jsonfile, outfile)

with open('temp/{}/intents/{}_usersays_en.json'.format(FILE_NAME, intent_name), 'w') as outfile:
    json.dump(intent_usersays_jsonfile, outfile)

# then process the next intent

In [16]:
# "project": "newagent-fcefe" may be an issue
agent_json = '''{
  "description": "",
  "language": "en",
  "disableInteractionLogs": false,
  "disableStackdriverLogs": true,
  "googleAssistant": {
    "googleAssistantCompatible": false,
    "project": "newagent-fcefe",
    "welcomeIntentSignInRequired": false,
    "startIntents": [],
    "systemIntents": [],
    "endIntentIds": [],
    "oAuthLinking": {
      "required": false,
      "grantType": "AUTH_CODE_GRANT"
    },
    "voiceType": "MALE_1",
    "capabilities": [],
    "protocolVersion": "V2",
    "isDeviceAgent": false
  },
  "defaultTimezone": "Asia/Hong_Kong",
  "webhook": {
    "available": false,
    "useForDomains": false,
    "cloudFunctionsEnabled": false,
    "cloudFunctionsInitialized": false
  },
  "isPrivate": true,
  "customClassifierMode": "use.after",
  "mlMinConfidence": 0.3,
  "supportedLanguages": [],
  "onePlatformApiVersion": "v2",
  "analyzeQueryTextSentiment": false,
  "enabledKnowledgeBaseNames": [],
  "knowledgeServiceConfidenceAdjustment": -0.4,
  "dialogBuilderMode": false
}'''

In [17]:
os.system("rm template.zip")
os.system("cd temp/{}/ && zip -r ../{}.zip * -x *.DS_Store".format(FILE_NAME, FILE_NAME))

0

In [18]:
# convert this notebook into a python script
# and then remove the last few lines
get_ipython().system('jupyter nbconvert --to script csv-to-df-one-intent.ipynb')
os.system("ex -snc '$-11,$d|x' csv-to-df-one-intent.py")

[NbConvertApp] Converting notebook csv-to-df-one-intent.ipynb to script
[NbConvertApp] Writing 8823 bytes to csv-to-df-one-intent.py
sed: can't read : No such file or directory
